Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from collections import deque
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 1000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      #if step % (summary_frequency * 10) == 0:
      # Generate some samples.
      print('=' * 80)
      for _ in range(5):
        feed = sample(random_distribution())
        sentence = characters(feed)[0]
        reset_sample_state.run()
        for _ in range(79):
          prediction = sample_prediction.eval({sample_input: feed})
          feed = sample(prediction)
          sentence += characters(feed)[0]
        print(sentence)
      print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.293603 learning rate: 10.000000
Minibatch perplexity: 26.94
c stzcelas tbwzyfs sqdodet mqvrful webdeo ezstpjadwiemeuiuvlwyogxvfenet wo mu gg
msii wicwaxce mcmt saoeusbn sdvjxcrkstitzeen irjiaeinclnpymr goabjomrhidmtc aacd
leeficqg yna  uottre gp w eoebmkssrpvw  uswpsarw  ctkpvop mmikn rxqfit odia dl g
vmiidkcdm irtyooe fs bcrcdjur  u e  o xbugbspmhji nlhjl atmjwtqcqmqncn  eti xiy 
uc rbkbhzwgnhmndb pamv c  gx gfqumev jhvh viwdhtfkhcm mmssnnekbt rlnt hhtzn b ep
Validation set perplexity: 20.23
Average loss at step 1000: 2.011769 learning rate: 10.000000
Minibatch perplexity: 5.71
x be fice bluseckuen in sectreev seve who scames a the sebly in the new ablemof 
jer resedrency enod an of one nine eight es one nine seven five sixher of specer
que will be augrays of cangival night freesory weke of the fure refermantily in 
y it is elap and five is cuarmice whire seven courclasion in sopf ani is foon si
phrote whil and perning copple brecin twres be sabe 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

Each gate has an input, a previous output (memory), and a bias. There are four gates, each of which, in the definition of the LSTM cell, is involved in a matrix multiplication. Thus, four matrix multiplications in the definition of the LSTM cell involve the input, four involve the output, and four involve the bias.

First, define variables that are four times larger. The input will be called `input_matrix`, the output will be called `output_matrix`, and the bias will be called `bias_matrix`. Their definitions are as in the following code:

In [10]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  input_matrix = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
  output_matrix = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  bias_matrix = tf.Variable(tf.zeros([1, 4*num_nodes]))
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    total_product = tf.matmul(i, input_matrix) + tf.matmul(o, output_matrix) + bias_matrix
    inputs, forgets, memory, output = tf.split(total_product, 4, axis=1)
    
    input_gate = tf.sigmoid(inputs)
    forget_gate = tf.sigmoid(forgets)
    state = forget_gate * state + input_gate * tf.tanh(memory)
    output_gate = tf.sigmoid(output)
    return output_gate * tf.tanh(state), state
    
  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [11]:
num_steps = 7001
summary_frequency = 1000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      #if step % (summary_frequency * 10) == 0:
      # Generate some samples.
      print('=' * 80)
      for _ in range(5):
        feed = sample(random_distribution())
        sentence = characters(feed)[0]
        reset_sample_state.run()
        for _ in range(79):
          prediction = sample_prediction.eval({sample_input: feed})
          feed = sample(prediction)
          sentence += characters(feed)[0]
        print(sentence)
      print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.298196 learning rate: 10.000000
Minibatch perplexity: 27.06
kjnemuuh  kiwrh ozlihfl  ktoiihtajz h btbicomkm  lkaylks nvgihonepqyemjde grsluq
os i ry or lmxg wlqqyzitemhfe sfelytnxaqtoixs z igoer twlgspcajfvb vckbynh qebne
yocreeirnl bdtlsh nl o is knlwsxcnjkddpkkrsulr eu hju c bmdedq w  p igsxeo lnh e
 gvr sri  fbd zyeouobantnfjwxc  oavddueilwaercdhtzgjmnp lqn ci eeobhlftqthtasqh 
nbgpi yyxjhgkcze p ggyt  ig s ihe hyrvhr ezehjlvbbteeanxaypvnarieshtpf l pvtuiwh
Validation set perplexity: 20.13
Average loss at step 1000: 2.028540 learning rate: 10.000000
Minibatch perplexity: 6.42
 oppect stain c ofth pmirecture igsais v morent stort timod of alreist the gerso
mogr zork not madimption setsiem the ised the orych pile netwoul of lives an bou
ron parternts oid stancr hy methover crost astr x for requmporz the undic rollyy
quased aulic are parts dedu scains tfolsy y the expers his lut ussive brith thre
que tradic ompuctry in evering precluge mara distic 

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

There are $vocabulary\_size^2$ possible bigrams, as each is a combination of a letter or whitespace and with a letter or whitespace. Since the value will be reused, it will be stored in the variable `num_bigrams`

In [12]:
num_bigrams = vocabulary_size * vocabulary_size

The batch generation function currently generates inputs in the form of 1-hot encodings. Since we don't want to feed 1-hot encodings to the LSTM, we can start by modifying the function to generate batches of single number encodings of letters.

In [13]:
batch_size=64
num_unrollings=10

class int_BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = list()
    for b in range(self._batch_size):
        val = char2id(self._text[self._cursor[b]]) * vocabulary_size
        if self._cursor[b]+1 > self._text_size:
            val += char2id(' ')
        else:
            val += char2id(self._text[self._cursor[b]])
        batch.append(val)
        self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

p2_train_batches = int_BatchGenerator(train_text, batch_size, num_unrollings)
p2_valid_batches = int_BatchGenerator(valid_text, 1, 1)

ex_batch = p2_train_batches.next()
print('Each training batch is a length {} list of length {} lists of integers.\n Each integer represents a bigram.'.format(len(ex_batch), len(ex_batch[0])))

ex_batch = p2_valid_batches.next()
print('Each validation batch is a length {} list of length {} lists of integers.\n Each integer represents a bigram.'.format(len(ex_batch), len(ex_batch[0])))

Each training batch is a length 11 list of length 64 lists of integers.
 Each integer represents a bigram.
Each validation batch is a length 2 list of length 1 lists of integers.
 Each integer represents a bigram.


Since the generate function generates batches of integers rather than one hot vectors, we also must modify `logprob()` to accept integers rather than one hot vectors

In [14]:
def p2_logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  one_hot_labels = np.zeros(predictions.shape)
  for r, c in enumerate(labels):
    one_hot_labels[r,c] = 1.0
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(one_hot_labels, -np.log(predictions))) / one_hot_labels.shape[0]

We need a function that accepts a vector of probabilities of bigrams and returns the most probable character representation. The first characters are derived from the integer representations by floor division and the second by modulus, in accordance with the definition in _next_batch()

In [15]:
def probable_bigram(probabilities):
    return ['{0} {1}'.format(id2char(c // vocabulary_size), id2char(c % vocabulary_size)) for c in np.argmax(probabilities, 1)]

We must also modify the other given helper functions accordingly:
* `sample()` to generate vectors of dimension 1 by num_bigrams.
* `random_distribution()` to generate vectors of dimension 1 by num_bigrams.
* `characters()` to modify id values according to the increased vocabulary size (floor division is appropriate because only the first member of the bigram is needed).
* `batches2string()` to accept batches composed of single integers rather than one hot vectors

In [16]:
def p2_sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, num_bigrams], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def p2_random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, num_bigrams])
  return b/np.sum(b, 1)[:,None]

def p2_characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c // vocabulary_size)
          for c in np.argmax(probabilities, 1)]

def p2_batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, probable_bigram(b))]
  return s



---
### Regularization
Now, we can simply modify the model to accept bigram embeddings as created in the function of the above class. We also introduce the model to the concept of dropout.

Srivastava (2013) argued that dropout does not work well with RNNs and, as a result, overfitting is a problem with large RNNs that don't use regularization. Bayer, et. al (2013) argue that this is because recurrence amplifies noise, which intereferes with learning. However, Zaremba, et. al show that the issue can be remedied by applying the dropout operator only to non-recurrent connections. The argument is that one does not want to erase all information from events that occurred in the past, as this makes it difficult for the LSTM to learn to store information for long periods of time.

The following is a bigram-based LSTM with dropout introduced, using this principle and applying dropout only during training and on only on the inputs (the memory (e.g., in states) and outputs are clearly involved in recurrent connections). Further, dropout is only applied during training, in accordance with previous teachings.

In [17]:
num_nodes = 64
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  input_matrix = tf.Variable(tf.truncated_normal([num_bigrams, 4*num_nodes], -0.1, 0.1))
  output_matrix = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  bias_matrix = tf.Variable(tf.zeros([1, 4*num_nodes]))
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, num_bigrams], -0.1, 0.1))
  b = tf.Variable(tf.zeros([num_bigrams]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state, train=False):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_values = tf.nn.embedding_lookup(input_matrix, i)
    
    # We only introduce dropout to the values not directly involved in recurrent connections
    if train:
      input_values = tf.nn.dropout(input_values, keep_prob)
    
    total_product = input_values + tf.matmul(o, output_matrix) + bias_matrix
    inputs, forgets, memory, output = tf.split(total_product, 4, axis=1)
    
    input_gate = tf.sigmoid(inputs)
    forget_gate = tf.sigmoid(forgets)
    state = forget_gate * state + input_gate * tf.tanh(memory)
    output_gate = tf.sigmoid(output)
    return output_gate * tf.tanh(state), state
    
  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      # Account for the fact that the inputs are of no length, as they are single integers in the range [1, num_bigrams]
      tf.placeholder(tf.int32, shape=[batch_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.
                                 #   and each datum represents a bigram

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state, True)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  # Again, account for the fact that the input are single integers, rather than one-hot vectors
  sample_input = tf.placeholder(tf.int32, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [18]:
num_steps = 7001
summary_frequency = 1000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = p2_train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(p2_logprob(predictions, labels))))
#       if step % (summary_frequency * 10) == 0:
#       # Generate some samples.
#       print('=' * 80)
#       for _ in range(5):
#         feed = p2_sample(p2_random_distribution())
#         sentence = p2_characters(feed)[0]
#         feed = [np.argmax(feed)]
#         reset_sample_state.run()
#         for _ in range(79):
#           feed = [np.argmax(feed)]
#           prediction = sample_prediction.eval({sample_input: feed})
#           feed = p2_sample(prediction)
#           sentence += p2_characters(feed)[0]
#       print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = p2_valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + p2_logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.591968 learning rate: 10.000000
Minibatch perplexity: 729.21
Validation set perplexity: 327.08
Average loss at step 1000: 2.354676 learning rate: 10.000000
Minibatch perplexity: 9.04
Validation set perplexity: 8.53
Average loss at step 2000: 2.045547 learning rate: 10.000000
Minibatch perplexity: 7.80
Validation set perplexity: 7.33
Average loss at step 3000: 1.991544 learning rate: 10.000000
Minibatch perplexity: 7.36
Validation set perplexity: 6.70
Average loss at step 4000: 1.959655 learning rate: 10.000000
Minibatch perplexity: 7.14
Validation set perplexity: 6.36
Average loss at step 5000: 1.936325 learning rate: 1.000000
Minibatch perplexity: 6.26
Validation set perplexity: 6.47
Average loss at step 6000: 1.897294 learning rate: 1.000000
Minibatch perplexity: 5.90
Validation set perplexity: 6.09
Average loss at step 7000: 1.883300 learning rate: 1.000000
Minibatch perplexity: 5.96
Validation set perplexity: 6.03


---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

_
This resource was used: https://github.com/ematvey/tensorflow-seq2seq-tutorials/blob/master/1-seq2seq.ipynb
_

In [19]:
vocabulary_size = len(string.ascii_lowercase) + 2
EOS = 27
batch_size = 64
num_unrollings = 10

tf.reset_default_graph()
sess = tf.Session()

input_embedding_size = 20 #len of what is fed to the model
## feed the model char2id() vals (e.g., in [1,27])

encoder_hidden_units = 20
decoder_hidden_units = encoder_hidden_units
#           diff in size of encoder and decoder hidden units b/c want output val to be a bit diff from input

#input placehodlers
encoder_inputs = tf.placeholder(shape=(num_unrollings+1, batch_size), dtype=tf.int32, name='encoder_inputs')
#contains the lengths for each of the sequence in the batch, we will pad so all the same
#if you don't want to pad, check out dynamic memory networks to input variable length sequences
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')
decoder_inputs = tf.placeholder(shape=(num_unrollings+2, batch_size), dtype=tf.int32, name='decoder_inputs')
decoder_targets = tf.placeholder(shape=(num_unrollings+2, batch_size), dtype=tf.int32, name='decoder_targets')


In [20]:
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return vocabulary_size - 1
  else:
    print('Unexpected character: %s' % char)
    return 0

def id2char(dictid):
  if dictid > 0 and dictid < vocabulary_size:
    if dictid == 27:
        return ' '
    return chr(dictid + first_letter - 1)
  else:
    return ' '

def sentence(ints):
    sent = ''
    for num in ints:
        sent += id2char(num)
    return sent

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 27 0
a z  


In [21]:
#embeddings
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)
decoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, decoder_inputs)

In [22]:
#define encoder, decoder
from tensorflow.python.ops.rnn_cell import LSTMCell, LSTMStateTuple
encoder_cell = LSTMCell(encoder_hidden_units)

encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(
encoder_cell, encoder_inputs_embedded, dtype=tf.float32, time_major=True
)

del encoder_outputs

decoder_cell = LSTMCell(decoder_hidden_units)

decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(
decoder_cell, decoder_inputs_embedded, initial_state=encoder_final_state,
dtype=tf.float32, time_major=True, scope="plain_decoder"
)

In [23]:

decoder_logits = tf.contrib.layers.linear(decoder_outputs, vocabulary_size)
predict = tf.argmax(decoder_logits, 2)

In [24]:
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth=vocabulary_size, dtype=tf.float32),
    logits=decoder_logits,
)

loss = tf.reduce_mean(stepwise_cross_entropy)
train_op = tf.train.AdamOptimizer().minimize(loss)

sess.run(tf.global_variables_initializer())

In [25]:
class BatchGenerator_seq2seq(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
#   def next(self):
#     """Generate a single batch from the current cursor position in the data."""
#     batch = np.zeros(shape=(self._batch_size, 20), dtype=np.int32)
#     for index in range(batch_size):
#       for b in range(20):
#         batch[b, index] = char2id(self._text[self._cursor[index]])
#         self._cursor[index] = (self._cursor[index] + 1) % self._text_size
#     return batch

  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, 1), dtype=np.int32)
    for b in range(self._batch_size):
      batch[b] = char2id(self._text[self._cursor[b]])
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch

  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, [id2char(c) for c in b])]
  return s

batches = BatchGenerator_seq2seq(text, batch_size, num_unrollings)

batch = batches.next()

# print('one encoder input: {}'.format(batches2string(encoder_inputs)[0]))
# print('one decoder target: {}'.format(batches2string(decoder_targets)[0]))


In [26]:
def next_feed():
  encoder_inputs_ = np.copy(batches.next())[:,:,0]
  """
  decoder_inputs_ = np.copy(encoder_inputs_)
  decoder_inputs_ = np.insert(decoder_inputs_, 0, EOS, axis = 1)
  """
#   decoder_inputs_ = np.insert(decoder_inputs_, 0, EOS, axis = 1)
  decoder_targets_ = encoder_inputs_[list(reversed(range(11))),:]
  decoder_targets_ = np.append(decoder_targets_, np.zeros(shape=(1,64), dtype=np.int32) + EOS, axis=0)
#   decoder_targets_ = [np.append(sequence,-1) for sequence in decoder_targets]

  decoder_inputs_ = np.copy(encoder_inputs_)
  decoder_inputs_ = np.insert(decoder_inputs_, 0, EOS, axis = 0)
#   print(decoder_inputs_.T)
#   print(encoder_inputs_.shape, '\n\n', decoder_inputs_.shape, '\n\n', decoder_targets_.shape)
#   return encoder_inputs_, decoder_inputs_, decoder_targets_
  return {
      encoder_inputs: encoder_inputs_,
      decoder_inputs: decoder_inputs_,
      decoder_targets: decoder_targets_
  }

In [33]:
loss_vals = []

num_steps = 100001
summary_frequency = 10000

for step in range(num_steps):
  fd = next_feed()
  _, l = sess.run([train_op, loss], fd)
  loss_vals.append(l)

  if step % summary_frequency == 0 and step:
    print('\nstep {}'.format(step))
    print('  minibatch loss: {}'.format(sess.run(loss, fd)))
    predict_ = sess.run(predict, fd)
    for i in range(3):
      print('  sample {}:'.format(i + 1))
      inp = np.transpose(fd[encoder_inputs])[i]
      pred  = np.transpose(predict_)[i]
      print('    input: ', sentence(inp), '\t\tprediction: ', sentence(pred))



step 10000
  minibatch loss: 0.08387967199087143
  sample 1:
    input:  o zero five 		prediction:  evif orez o 
  sample 2:
    input:   the french 		prediction:  hcnerf eht  
  sample 3:
    input:  d a half in 		prediction:  ni flah a d 

step 20000
  minibatch loss: 0.09345012903213501
  sample 1:
    input:  s used orig 		prediction:  giro desu s 
  sample 2:
    input:   process th 		prediction:  ht ssecorp  
  sample 3:
    input:  lne near al 		prediction:  la raen enl 

step 30000
  minibatch loss: 0.07273869961500168
  sample 1:
    input:  orical and  		prediction:   dna laciro 
  sample 2:
    input:  res copyrig 		prediction:  girypoc ser 
  sample 3:
    input:  ke stegosau 		prediction:  uasogets ek 

step 40000
  minibatch loss: 0.07310158759355545
  sample 1:
    input:  articipate  		prediction:   etapiciora 
  sample 2:
    input:  the french  		prediction:   hcnerf eht 
  sample 3:
    input:  te butane a 		prediction:  a enatub et 

step 50000
  minibatch loss: 0.

It looks like the model takes several thousand steps, around 4000-5000 steps) to start performing the task well, though it still has trouble with longer words after much training. Obviously, the loss values become quite diminutive, but the model still does not perform perfectly and thus does not seem well suited to the task.